# AutoARIMA on Stock Prices

In [55]:
# Importing Libraries
import pandas as pd
import numpy as np
import itertools
from statistics import mean, median
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import AutoARIMA
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from datetime import date, timedelta
import yfinance as yf

In [215]:
# Getting the date five years ago to download the current timeframe
years = (date.today() - timedelta(weeks=300)).strftime("%Y-%m-%d")

# Stocks to analyze
stocks = ['CCL', 'OXY', 'KSS', 'COTY']

# Getting the data for multiple stocks
df = yf.download(stocks, start=years)

print("Rows in DataFrame: ", df.shape[0])

[*********************100%***********************]  4 of 4 completed
Rows in DataFrame:  1447


In [216]:
# Storing the dataframes in a dictionary
stock_df = {}

for col in set(df.columns.get_level_values(0)):
    
    # Assigning the information (High, Low, etc.) for each stock in the dictionary
    stock_df[col] = df[col]

# Preprocessing Data

Scale the data using a logarithmic scale.  Also rounding the log result by 2 decimal points in order to reduce any unnecessary noise.

In [217]:
# Finding the log returns
stock_df['LogReturns'] = stock_df['Adj Close'].apply(np.log).diff().dropna()

# Trying out Moving average
stock_df['Adj Close'] = stock_df['Adj Close'].rolling(15).mean().dropna()

# Logarithmic scaling of the data and rounding the result
stock_df['LogClose'] = stock_df['Adj Close'].apply(np.log).apply(lambda x: round(x, 2))

# Visualizing the Data

In [218]:
px.line(stock_df['Adj Close'], 
        x=stock_df['Adj Close'].index, 
        y=stock_df['Adj Close'].columns,
        labels={'variable': 'Stock',
                'value': 'Price'},
        title='Adj Close')


In [219]:
px.line(stock_df['LogClose'], 
        x=stock_df['LogClose'].index, 
        y=stock_df['LogClose'].columns,
        labels={'variable': 'Stock',
                'value': 'Log Price'},
        title='Log of Closing Prices')

## Optimum Parameter Search Function

In [220]:
opt_param = AutoARIMA(start_p=0, start_q=0,
                      start_P=0, start_Q=0,
                      max_p=8, max_q=8,
                      max_P=5, max_Q=5,
                      error_action='ignore',
                      information_criterion='bic',
                      suppress_warnings=True)

for stock in tqdm(stocks):

    opt_param.fit(stock_df['LogClose'][stock])

    print(f'Summary for {stock}', '--'*20)
    display(opt_param.summary())

Summary for CCL ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1433
Model:               SARIMAX(1, 2, 1)   Log Likelihood                5289.499
Date:                Tue, 08 Sep 2020   AIC                         -10572.998
Time:                        20:33:40   BIC                         -10557.199
Sample:                             0   HQIC                        -10567.098
                               - 1433                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.2330      0.035     -6.603      0.000      -0.302      -0.164
ma.L1         -0.4427      0.028    -15.697      0.000      -0.498      -0.387
sigma2      3.604e-05   1.03e-06     34.985      0.000     3.4e-05    3.81e-05
===================================================================================
Ljung-Box (Q):                       98.19   Jarque-Bera (JB):              1165.19
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.95   Skew:                             0.62
Prob(H) (two-sided):                  0.00   Kurtosis:                         7.25
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for OXY ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1433
Model:               SARIMAX(2, 2, 0)   Log Likelihood                5248.752
Date:                Tue, 08 Sep 2020   AIC                         -10491.505
Time:                        20:33:42   BIC                         -10475.706
Sample:                             0   HQIC                        -10485.605
                               - 1433                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.6204      0.016    -38.396      0.000      -0.652      -0.589
ar.L2         -0.2130      0.018    -11.674      0.000      -0.249      -0.177
sigma2      3.814e-05   6.36e-07     59.935      0.000    3.69e-05    3.94e-05
===================================================================================
Ljung-Box (Q):                      119.33   Jarque-Bera (JB):             14454.24
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               2.50   Skew:                             1.15
Prob(H) (two-sided):                  0.00   Kurtosis:                        18.40
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for KSS ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1433
Model:               SARIMAX(3, 1, 0)   Log Likelihood                5292.372
Date:                Tue, 08 Sep 2020   AIC                         -10576.745
Time:                        20:33:49   BIC                         -10555.677
Sample:                             0   HQIC                        -10568.878
                               - 1433                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2762      0.022     12.480      0.000       0.233       0.320
ar.L2          0.3619      0.022     16.623      0.000       0.319       0.405
ar.L3          0.2329      0.020     11.581      0.000       0.193       0.272
sigma2      3.606e-05   1.16e-06     31.017      0.000    3.38e-05    3.83e-05
===================================================================================
Ljung-Box (Q):                      116.30   Jarque-Bera (JB):               146.29
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.90   Skew:                             0.16
Prob(H) (two-sided):                  0.00   Kurtosis:                         4.53
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for COTY ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1433
Model:               SARIMAX(0, 2, 2)   Log Likelihood                5185.326
Date:                Tue, 08 Sep 2020   AIC                         -10364.651
Time:                        20:33:52   BIC                         -10348.853
Sample:                             0   HQIC                        -10358.752
                               - 1433                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.6238      0.020    -31.034      0.000      -0.663      -0.584
ma.L2          0.1652      0.022      7.584      0.000       0.122       0.208
sigma2      4.163e-05   1.32e-06     31.437      0.000     3.9e-05    4.42e-05
===================================================================================
Ljung-Box (Q):                      205.24   Jarque-Bera (JB):               240.25
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.95   Skew:                             0.48
Prob(H) (two-sided):                  0.00   Kurtosis:                         4.76
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

# Using the ARIMA Model
Using the price history from the past N days to make predictions

In [ ]:
# Days in the past to train on
days_to_train = 365 

# Days in the future to predict
days_to_predict = 10

# Establishing a new DFs for predictions
stock_df['Predictions'] = pd.DataFrame(index=stock_df['LogClose'].index,
                                       columns=stock_df['LogClose'].columns)

# Iterate through each stock
for stock in tqdm(stocks):
    
    # Training a model for each day and getting predictions
    for day in tqdm(range(days_to_train, stock_df['LogClose'].shape[0]-days_to_predict, days_to_predict)):

        # Data to use, containing rolling amount of past days
        training = stock_df['LogClose'][stock].iloc[day-days_to_train:day].dropna()

        # Finding the best parameters
        model    = AutoARIMA(start_p=0, start_q=0,
                             start_P=0, start_Q=0,
                             max_p=8, max_q=8,
                             max_P=5, max_Q=5,
                             error_action='ignore',
                             information_criterion='aic',
                             suppress_warnings=True)

        # Getting predictions for the optimum parameters by fitting to the training set            
        forecast = model.fit_predict(training,
                                     n_periods=days_to_predict)

        # Getting the overall average prediction for the next N days
        stock_df['Predictions'][stock].iloc[day:day+days_to_predict] = np.exp(forecast)


# Predictions vs Actual Values

In [201]:
# Shift ahead by 1 to compare the actual values to the predictions
pred_df = stock_df['Predictions'].shift(1).astype(float).dropna()

pred_df

,CCL,COTY,KSS,OXY
Date,,,,
2016-05-24,43.816042,21.977078,28.789191,60.340288
2016-05-25,43.816042,21.977078,28.789191,60.340288
2016-05-26,43.816042,21.977078,28.789191,60.340288
2016-05-27,43.816042,21.977078,28.789191,60.340288
2016-05-31,43.816042,21.977078,28.789191,60.340288
...,...,...,...,...
2020-08-31,14.585093,3.819044,18.915846,13.463738
2020-09-01,14.585093,3.819044,18.915846,13.463738
2020-09-02,14.585093,3.819044,18.915846,13.463738


## Plotting the Predictions
Comparing the actual values with the predictions

In [202]:
for stock in stocks:
    
    fig = go.Figure()
    
    # Plotting the actual moving average values
    fig.add_trace(go.Scatter(x=pred_df.index,
                             y=stock_df['Adj Close'][stock].loc[pred_df.index],
                             name='Actual Adj Close',
                             mode='lines'))
    
    # Plotting the predicted moving average value
    fig.add_trace(go.Scatter(x=pred_df.index,
                             y=pred_df[stock],
                             name='Predicted Adj Close',
                             mode='lines'))
    
    # Setting the labels
    fig.update_layout(title=f'Predicting the Average Adj Close for the Next {days_to_predict} days for {stock}',
                      xaxis_title='Date',
                      yaxis_title='Prices')
    
    fig.show()

## Evaluation Metric

In [203]:
for stock in stocks:
    
    # Finding the root mean squared error
    rmse = mean_squared_error(stock_df['Adj Close'][stock].loc[pred_df.index],
                              pred_df[stock],
                              squared=False)

    print(f"On average, the model is off by ${round(rmse, 2)} for {stock}\n")

On average, the model is off by $2.55 for CCL

On average, the model is off by $2.87 for OXY

On average, the model is off by $3.27 for KSS

On average, the model is off by $0.95 for COTY



# Trading Signal
Turning the model into a Trading Signal

In [204]:
def get_positions(difference, thres=3, short=True):
    """
    Compares the percentage difference between actual values and the respective predictions.
    
    Returns the decision or positions to long or short based on the difference.
    
    Optional: shorting in addition to buying
    """
    
    if difference > thres/100:
        
        return 1
    
    
    elif short and difference < -thres/100:
        
        return -1
    
    
    else:
        
        return 0

### Creating a Trading DF
__Note:__ _On Preventing Lookahead Bias_

For example, if the model is ran after hours and a position is established on the next day's opening, then a shift ahead of 1 is ok.  But if a position is established on the next day, near the close, then it needs to be shifted ahead by 2, because the newly established position missed any gains or losses that day.  These are due to the fact that gains or losses in the day are determined when a trade is entered.

(This can also determine how long the predicted forecast remains valid.)

In [205]:
# Creating a DF for trading the model
trade_df = {}

# Getting the percentage difference between the predictions and the actual values
trade_df['PercentDiff'] = (stock_df['Predictions'].dropna() / stock_df['Adj Close'].loc[stock_df['Predictions'].dropna().index]) - 1

# Getting positions
trade_df['Positions'] = trade_df['PercentDiff'].applymap(lambda x: get_positions(x, 
                                                                                 thres=1, 
                                                                                 short=True) / len(stocks))

# Preventing lookahead bias
trade_df['Positions'] = trade_df['Positions'].shift(2).dropna()

# Getting Log Returns
trade_df['LogReturns'] = stock_df['LogReturns'].loc[trade_df['Positions'].index]                                    
    
display(trade_df['PercentDiff'])
display(trade_df['Positions'])

,CCL,COTY,KSS,OXY
Date,,,,
2016-05-23,-0.000846441,-0.0199294,0.00966155,0.00153558
2016-05-24,-0.0189306,-0.0304596,0.0153677,-0.0149109
2016-05-25,-0.00212826,-0.0187874,0.00122194,-0.0155575
2016-05-26,0.0406467,-0.0319456,0.0102292,-0.0147816
2016-05-27,0.0190202,-0.0378439,-0.00487807,-0.0152989
...,...,...,...,...
2020-08-28,-0.152522,0.0293918,-0.126289,0.025418
2020-08-31,-0.114982,0.066772,-0.114427,0.0568083
2020-09-01,-0.11659,0.0638004,-0.0979568,0.0788252


,CCL,COTY,KSS,OXY
Date,,,,
2016-05-25,0.00,-0.25,0.00,0.00
2016-05-26,-0.25,-0.25,0.25,-0.25
2016-05-27,0.00,-0.25,0.00,-0.25
2016-05-31,0.25,-0.25,0.25,-0.25
2016-06-01,0.25,-0.25,0.00,-0.25
...,...,...,...,...
2020-08-28,-0.25,0.00,-0.25,0.25
2020-08-31,-0.25,0.25,-0.25,0.25
2020-09-01,-0.25,0.25,-0.25,0.25


## Plotting the Positions

In [206]:
# Getting the number of positions
pos = trade_df['Positions'].apply(pd.value_counts)

# Plotting total positions
fig = px.bar(pos, 
             x=pos.index, 
             y=pos.columns,
             title='Total Positions',
             labels={'variable':'Stocks',
                      'value':'Count of Positions',
                      'index':'Position'})

fig.show()


# Calculating and Plotting the Potential Returns

## Returns on Each Individual Stock

In [207]:
# Calculating Returns by multiplying the positions by the log returns
returns = trade_df['Positions'] * trade_df['LogReturns']

# Calculating the performance as we take the cumulative sum of the returns and transform the values back to normal
performance = returns.cumsum().apply(np.exp)

# Plotting the performance per stock
px.line(performance,
        x=performance.index,
        y=performance.columns,
        title='Returns Per Stock Using ARIMA Forecast',
        labels={'variable':'Stocks',
                'value':'Returns'})

## Returns on the Overall Portfolio

In [208]:
# Returns for the portfolio
returns = (trade_df['Positions'] * trade_df['LogReturns']).sum(axis=1)

# Returns for SPY
spy = yf.download('SPY', start=returns.index[0])

spy = spy['Adj Close'].apply(np.log).diff().dropna().cumsum().apply(np.exp)

# Calculating the performance as we take the cumulative sum of the returns and transform the values back to normal
performance = returns.cumsum().apply(np.exp)

# Plotting the comparison between SPY returns and GARCH returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=spy.index,
                         y=spy,
                         name='SPY Returns',
                         mode='lines'))

fig.add_trace(go.Scatter(x=performance.index,
                         y=performance.values,
                         name='ARIMA Returns on Portfolio',
                         mode='lines'))

fig.update_layout(title='SPY vs ARIMA Overall Portfolio Returns',
                  xaxis_title='Date',
                  yaxis_title='Returns')

fig.show()

[*********************100%***********************]  1 of 1 completed
